In [1]:
pwd

'/home/wsuser/work'

In [2]:
imageSize = [299, 299]
#trainPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/training"
#testPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/testing"
trainPath = "/home/wsuser/work/preprocessed dataset/preprocessed dataset/training"
testPath = "/home/wsuser/work/preprocessed dataset/preprocessed dataset/testing"

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
type(train_datagen)

keras.preprocessing.image.ImageDataGenerator

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RzL20fskxy-Y1gjml2csOOZat7w24Bdi8TNvronExyp7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'diabeticfinal-donotdelete-pr-f5nzyd891ylukm'
object_key = 'preprocessed dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [7]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
pwd

'/home/wsuser/work'

In [9]:
import os
filenames=os.listdir('/home/wsuser/work/preprocessed dataset/preprocessed dataset/training')

In [11]:
training_set =train_datagen.flow_from_directory(trainPath, target_size = (299,299),batch_size = 4, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(testPath, target_size = (299,299),batch_size = 4, class_mode = 'categorical')

Found 137 images belonging to 5 classes.
Found 117 images belonging to 5 classes.


In [12]:
xception = Xception(input_shape = imageSize + [3], weights = 'imagenet', include_top = False)

83697664/83683744 [==============================] - 0s 0us/step


In [13]:
for layer in xception.layers:
  layer.trainable = False

x = Flatten()(xception.output)

prediction = Dense(5, activation = 'softmax')(x)

model = Model(inputs = xception.input, outputs = prediction)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
r = model.fit_generator(training_set, validation_data = test_set, epochs = 50, steps_per_epoch = len(training_set)//32, validation_steps = len(test_set)//32)

/tmp/wsuser/ipykernel_164/966271731.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(training_set, validation_data = test_set, epochs = 50, steps_per_epoch = len(training_set)//32, validation_steps = len(test_set)//32)


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 20.8216 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 13.3012 - accuracy: 0.7500
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 10.3953 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 16.1016 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 16.1712 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 2s 2s/step - loss: 22.0960 - accuracy: 0.2500
Epoch 7/50
1/1 [==============================] - 2s 2s/step - loss: 13.6296 - accuracy: 0.2500
Epoch 8/50
1/1 [==============================] - 2s 2s/step - loss: 21.5347 - accuracy: 0.2500
Epoch 9/50
1/1 [==============================] - 2s 2s/step - loss: 0.0067 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 2s 2s/step - loss: 1.9726 - accuracy: 0.7500
Epoch 11/50
1/1 [========================

In [18]:
model.save('final.h5')

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [19]:
!tar -zcvf diabetic-retinopathy-model_new.tgz final.h5

final.h5


In [20]:
ls

 diabetic-retinopathy-model_new.tgz   final.h5  'preprocessed dataset'/


In [21]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 14.2 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    'url':'https://us-south.ml.cloud.ibm.com',
    'apikey':'Nh5wxaW-n-x4wen51BeGt-gVRFumapUtikckfqs2T74u'
}

client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [26]:
space_uid=guid_from_space_name(client,'diabetic_retinopathy')
print("Space UID="+space_uid)

Space UID=5eb3dfc2-35d5-44b5-9689-6f2dd7dadb95


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
model_details = client.repository.store_model(model="diabetic-retinopathy-model_new.tgz", meta_props={
    client.repository.ModelMetaNames.NAME:"Diabetic_Retinopathy_Detection",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})